In [2]:
import pandas as pd
import os
import csv
import numpy as np
from scipy import stats
from matplotlib import pyplot as plt
%matplotlib

Using matplotlib backend: Qt5Agg


In [3]:
from scipy.ndimage.filters import median_filter

In [4]:
from Experimentos import Experimentos

[Errno 2] No such file or directory: '.\\Data\\5cc50C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\1cc54C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\25cc54C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\10cc60C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\25cc60C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\1cc59C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\5cc63C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\25cc62C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\1cc62C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\1cc24C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\10cc24C - flir.csv'


In [5]:
for i, exp in enumerate(Experimentos):
    print(i, exp, (exp.flir is not None))

0 5cc50C.csv False
1 1cc54C.csv False
2 25cc54C.csv False
3 10cc60C.csv False
4 25cc60C.csv False
5 1cc59C.csv False
6 1cc47C.csv True
7 1cc55C.csv True
8 25cc58C.csv True
9 25cc70C.csv True
10 10cc67C.csv True
11 10cc59C.csv True
12 5cc63C.csv False
13 5cc59C.csv True
14 10cc49C.csv True
15 25cc62C.csv False
16 1cc62C.csv False
17 5cc53C.csv True
18 5a25cc56C.csv True
19 5cc57C.csv True
20 25cc50C.csv True
21 10cc55C.csv True
22 1cc24C.csv False
23 10cc57C.csv True
24 10cc24C.csv False
25 5cc55C.csv True
26 5cc58C.csv True
27 1cc48C.csv True
28 1cc51C.csv True
29 5cc48C.csv True
30 10cc50C.csv True
31 1cc54,7C.csv True
32 1cc52C.csv True


In [6]:
for exp in Experimentos:
    if exp.flir is not None:
        try:
            exp.flir['area_m2'] = exp.flir['FLIR: area']/numeropixeles
            exp.flir['volumen_emitido'] = exp.param['caudal'][0]*exp.flir['FLIR: tiempo']
        except:
            continue
        volumen_total = kilos_manjar/rho_manjar
        exp.flir['volumen_emitido'][exp.flir['volumen_emitido'] > volumen_total] = volumen_total
        exp.flir['espesor'] = 3*exp.flir['volumen_emitido']/(2*exp.flir['area_m2'])

In [46]:
#FALTA: agregar rango de temperaturas a partir de archivo "resumen general experimentos":
#(max-min)/rangos = salto temp
#rangos=min+salto etc

In [ ]:
#FALTA: graficar tiempo en que prevalece cada rango de temperatura? y como relacionarlo a la velocidad

In [7]:
Experimentos[30].flir.head()

,FLIR: tiempo,FLIR: area,FLIR: rango 1,FLIR: rango 2,FLIR: rango 3,FLIR: rango 4,FLIR: rango 5,FLIR: rango 6,FLIR: rango 7,FLIR: rango 8,FLIR: rango 9
0,0,1666,0.587635,0.063625,0.055222,0.120648259,0.049819928,0.070828331,0.027611044,0.024610,0.000000
1,1,2254,0.478261,0.053239,0.035492,0.066104703,0.022626442,0.068322981,0.20851819,0.067436,0.000000
2,2,3051,0.443461,0.043592,0.025565,0.033431662,0.018354638,0.033759423,0.106850213,0.234677,0.060308
3,3,3677,0.403590,0.039162,0.019581,0.014413924,0.010606473,0.027740005,0.071797661,0.221648,0.191460
4,4,3923,0.305379,0.037216,0.013765,0.01172572,0.011470813,0.023196533,0.071119042,0.253887,0.272241


In [9]:
def rango_temperatura(temp):
    if temp < 40:
        return 1
    if temp < 45:
        return 2
    if temp < 50:
        return 3
    if temp < 55:
        return 4
    if temp < 60:
        return 5
    if temp < 65:
        return 6
    if temp < 70:
        return 7

In [10]:
#GENERA PATCHES DE COLORES SEGUN TEMPERATURA
import matplotlib.patches as mpatches

pat1 = mpatches.Patch(color='C1', label='temp < 40')
pat2 = mpatches.Patch(color='C2', label='temp < 45')
pat3 = mpatches.Patch(color='C3', label='temp < 50')
pat4 = mpatches.Patch(color='C4', label='temp < 55')
pat5 = mpatches.Patch(color='C5', label='temp < 60')
pat6 = mpatches.Patch(color='C6', label='temp < 65')
pat7 = mpatches.Patch(color='C7', label='temp < 70')

numerocolores = 7
colors = ['pat{}'.format(i+1) for i in range(numerocolores)]

print(colors)

['pat1', 'pat2', 'pat3', 'pat4', 'pat5', 'pat6', 'pat7']


In [37]:
#LISTA para ESPESOR vs TIEMPO
listaporcaudal = []

for exp in Experimentos:
    if exp.flir is not None:
        try:
            holder = {} #es un diccionario 
            holder['nombre'] = exp.param['NOMBRE'][0]#solo el valor de la primera fila
            holder['espesor'] = exp.flir['espesor']#toda la columna
            holder['tiempo'] = exp.flir['FLIR: tiempo']#toda la columna
            holder['caudal'] = exp.param['caudal'][0]#solo el valor de la primera fila
            holder['angulo'] = exp.param['angulo inclinacion'][0]
            holder['uso'] = exp.param['uso manjar'][0]

            listaporcaudal.append(holder)
        except:
            continue

In [27]:
#crear GRUPOS segun CAUDAL
from itertools import groupby
count = 0

#key es la lista donde estan los diccionarios de exp y DF
#group es que se agrupa segun el valor de 'caudal' en este caso

for key, group in groupby(sorted(listaporcaudal, key=lambda x: x['caudal']), key=lambda x: x['caudal']):
    print(key)
    count += 1
    #for exp in group:
        #muestra el nombre del grupo y los elementos que tiene
        #print(exp['nombre']) 

1e-06
5e-06
1e-05
1.7e-05
2.5e-05


In [38]:
#ESPESOR vs TIEMPO agrupados por caudales
fig = plt.figure()
axes = [fig.add_subplot(3,2,i+1) for i in range(count)]
counter = 0

for key, group in groupby(sorted(listaporcaudal, key=lambda x: x['caudal']), key=lambda x: x['caudal']):
    print(key)
    ax = axes[counter]
    #ax.set_xlim(minimoejelargo, maximoejelargo)
    #ax.set_ylim(0, maximoejeespesor)
    ax.set_title('{}cc'.format(key*1000000))
    counter += 1
    linea1= ax.plot([],[], label='15°', linestyle='--', color='black')
    linea2= ax.plot([],[], label='10°', color='black')
    
    for exp in group:
        #print("C{}".format(exp["rango temperatura"]))
        #if exp['uso'] == 1: 
            if exp['angulo'] == 15:
                ax.plot(exp['tiempo'], exp['espesor'], linestyle= '--')
            else:
                ax.plot(exp['tiempo'], exp['espesor'])
        
legend1 = plt.legend(handles=[pat1, pat2, pat3, pat4, pat5, pat6, pat7], bbox_to_anchor=(1.8, 0.5), loc=5)
plt.legend(bbox_to_anchor=(1.8, 0.5), loc=6)
plt.gca().add_artist(legend1)
fig.show()

1e-06
5e-06
1e-05
1.7e-05
2.5e-05


In [39]:
#LISTA para AREA vs TIEMPO
otralistaporcaudal = []

for exp in Experimentos:
    if exp.flir is not None:
        try:
            holder = {} #es un diccionario 
            holder['nombre'] = exp.param['NOMBRE'][0]#solo el valor de la primera fila
            holder['area'] = exp.flir['area_m2']#toda la columna
            holder['tiempo'] = exp.flir['FLIR: tiempo']#toda la columna
            holder['caudal'] = exp.param['caudal'][0]#solo el valor de la primera fila
            holder['angulo'] = exp.param['angulo inclinacion'][0]
            holder['uso'] = exp.param['uso manjar'][0]

            otralistaporcaudal.append(holder)
        except:
            continue

In [40]:
#AREA vs TIEMPO agrupados por caudales
fig = plt.figure()
axes = [fig.add_subplot(3,2,i+1) for i in range(count)]
counter = 0

for key, group in groupby(sorted(otralistaporcaudal, key=lambda x: x['caudal']), key=lambda x: x['caudal']):
    print(key)
    ax = axes[counter]
    #ax.set_xlim(minimoejelargo, maximoejelargo)
    #ax.set_ylim(0, maximoejeespesor)
    ax.set_title('{}cc'.format(key*1000000))
    counter += 1
    
    for exp in group:
        ax.plot(exp['tiempo'], exp['area'])
        
legend1 = plt.legend(handles=[pat1, pat2, pat3, pat4, pat5, pat6, pat7], bbox_to_anchor=(1.8, 0.5), loc=5)
plt.legend(bbox_to_anchor=(1.8, 0.5), loc=6)
plt.gca().add_artist(legend1)
fig.show()

1e-06
5e-06
1e-05
1.7e-05
2.5e-05
